In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import pickle
import numpy as np
import multiprocessing as mp
from IPython.display import display, Image, SVG

from pydrake.geometry import Role, SceneGraph
from pydrake.geometry.optimization import (
    HPolyhedron,
    Hyperellipsoid,
    IrisInConfigurationSpace,
    IrisOptions
)
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.gurobi import GurobiSolver
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.meshcat_visualizer import ConnectMeshcatVisualizer
from pydrake.systems.primitives import TrajectorySource
from pydrake.systems.rendering import MultibodyPositionToGeometryPose

from meshcat import Visualizer
import meshcat.geometry as g

from comparison.helpers import make_traj
from examples.bimanual.helpers import *
from gcs.linear import LinearGCS

GurobiSolver.AcquireLicense()
MosekSolver.AcquireLicense()

In [ ]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

In [ ]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add("gcs", os.path.dirname("./package.xml"))

directives_file = "./models/bimanual_iiwa.yaml"
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)
[iiwa_1, wsg_1, iiwa_2, wsg_2, shelf, bin_1, bin_2, table] = models


plant.Finalize()

viz_role = Role.kIllustration
# viz_role = Role.kProximity
visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url,
                                      delete_prefix_on_load=False, role=viz_role)

diagram = builder.Build()

q0 = [0.0, -0.2, 0, -1.2, 0, 1.6, 0.0]
index = 0
for joint_index in plant.GetJointIndices(iiwa_1.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1
index = 0
for joint_index in plant.GetJointIndices(iiwa_2.model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1
        
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
sg_context = scene_graph.GetMyMutableContextFromRoot(context)
diagram.Publish(context)

In [ ]:
iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.02
iris_options.enable_ibex = False
CORE_CNT = int(mp.cpu_count()/2) # you may edit this

filterCollsionGeometry(scene_graph, sg_context)

seed_points = getConfigurationSeeds()

In [ ]:
context_clones = [context.Clone() for _ in range(len(seed_points))]

def calcRegion(ii, seed):
    start_time = time.time()
    context = context_clones[ii]
    plant_context = plant.GetMyMutableContextFromRoot(context)
    plant.SetPositions(plant_context, seed)
    
    hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
    print("Seed:", ii, "\tTime:", np.round((time.time() - start_time)/60., 4),
          "minutes.\tFaces", len(hpoly.b()), flush=True)
    return hpoly

def generateRegions(seed_points):
    seeds = list(seed_points.values()) if type(seed_points) is dict else seed_points
    regions = []
    loop_time = time.time()
    with mp.Pool(processes = CORE_CNT) as pool:
        regions = pool.starmap(calcRegion, [[ii, seed] for ii, seed in enumerate(seeds)])

    print("Loop time:", time.time() - loop_time)
    
    if type(seed_points) is dict:
        return dict(list(zip(seed_points.keys(), regions)))

    return regions
    
# Takes approximately 30 minutes to generate on 10 cores (3 hours if run serially)
regions = generateRegions(seed_points)

with open('./data/bimanual/iris_regions.reg', 'wb') as f:
    pickle.dump(regions,f)

In [ ]:
with open('data/bimanual/iris_regions.reg', 'rb') as f:
    regions = pickle.load(f)

In [ ]:
gcs = LinearGCS(regions)
display(Image(gcs.VisualizeGraph("png")))

# Demonstration

In [ ]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()
    
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add("gcs", os.path.dirname("./package.xml"))

    directives_file = "./models/bimanual_iiwa.yaml"
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    [iiwa_1, wsg_1, iiwa_2, wsg_2, shelf, binR, binL, table] =  models
    
    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    if type(traj) is list:
        traj_system = builder.AddSystem(VectorTrajectorySource(traj))
        end_time = np.sum([t.end_time() for t in traj])
    else:
        traj_system = builder.AddSystem(TrajectorySource(traj))
        end_time = traj.end_time()
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())
    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()
    simulator = Simulator(vis_diagram)
    
    plant_context = plant.CreateDefaultContext()
    c_list_rgb = [[i/255 for i in (0, 0, 255, 255)],[i/255 for i in (255, 191, 0, 255)],[i/255 for i in (255, 64, 0, 255)]]
    iiwa1_X = []
    iiwa2_X = []
    if type(traj) is list:
        for t in traj:
            q_waypoints = t.vector_values(np.linspace(t.start_time(), t.end_time(), 1000))
            for ii in range(q_waypoints.shape[1]):
                plant.SetPositions(plant_context, q_waypoints[:, ii])
                iiwa1_X.append(plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg_1.model_instance)))
                iiwa2_X.append(plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg_2.model_instance)))

        vertices = list(map(lambda X: X.translation(), iiwa1_X))
        colors = [np.array(c_list_rgb[0]) for _ in range(len(iiwa1_X))]
        vertices = np.stack(vertices).astype(np.float32).T
        colors = np.array(colors).astype(np.float32).T
        vis["paths"]["iiwa_1"].set_object(g.Points(g.PointsGeometry(vertices, color=colors),
                                        g.PointsMaterial(size=0.015)))
        vertices = list(map(lambda X: X.translation(), iiwa2_X))
        colors = [np.array(c_list_rgb[0]) for _ in range(len(iiwa2_X))]
        vertices = np.stack(vertices).astype(np.float32).T
        colors = np.array(colors).astype(np.float32).T
        vis["paths"]["iiwa_2"].set_object(g.Points(g.PointsGeometry(vertices, color=colors),
                                        g.PointsMaterial(size=0.015)))
    
    meshcat.start_recording()
    simulator.AdvanceTo(end_time)
    meshcat.publish_recording()

In [ ]:
demo = getDemoConfigurations()

In [ ]:
trajectories, run_time = getBezierGcsPath(plant, regions, demo, 3, 1)
print("Trajectory generation took", np.round(run_time, 4), "seconds of optimizer time.")
visualize_trajectory(trajectories)

with open ("data/bimanual/bezier_trajectory.html", "w") as f:
   f.write(vis.static_html())

In [ ]:
from pydrake.common import FindResourceOrThrow
from comparison.helpers import lower_alpha

def generate_segment_pics(traj, segment):
    builder = DiagramBuilder()
    
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    parser = Parser(plant, scene_graph)
    parser.package_map().Add("mp-gcs", os.path.dirname("./package.xml"))
    directives_file = "./models/bimanual_iiwa.yaml"
    iiwa_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/urdf/iiwa14_spheres_collision.urdf")
    wsg_file = "./models/schunk_wsg_50_welded_fingers.sdf"
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    [iiwa1_start, wsg1_start, iiwa2_start, wsg2_start, shelf, binR, binL, table] =  models
    
    
    iiwa1_goal = parser.AddModelFromFile(iiwa_file, "iiwa1_goal")
    wsg1_goal = parser.AddModelFromFile(wsg_file, "wsg1_goal")
    iiwa2_goal = parser.AddModelFromFile(iiwa_file, "iiwa2_goal")
    wsg2_goal = parser.AddModelFromFile(wsg_file, "wsg2_goal")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", iiwa1_goal), RigidTransform())
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa1_goal), plant.GetFrameByName("body", wsg1_goal),
                     RigidTransform(rpy=RollPitchYaw([np.pi/2., 0, np.pi/2]), p=[0, 0, 0.114]))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", iiwa2_goal), RigidTransform([0, 0.5, 0]))
    plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa2_goal), plant.GetFrameByName("body", wsg2_goal),
                     RigidTransform(rpy=RollPitchYaw([np.pi/2., 0, np.pi/2]), p=[0, 0, 0.114]))
    
    arm_models = [iiwa1_start.model_instance, wsg1_start.model_instance,
                  iiwa2_start.model_instance, wsg2_start.model_instance,
                  iiwa1_goal, wsg1_goal, iiwa2_goal, wsg2_goal]
    lower_alpha(plant, scene_graph.model_inspector(), arm_models, 0.4, scene_graph)
    
    plant.Finalize()
    
    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    diagram = builder.Build()

    if type(traj) is not list:
        traj = [traj]

    meshcat.load()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    sg_context = scene_graph.GetMyMutableContextFromRoot(context)
    
    t = traj[segment]
        
    iiwa1_X = []
    iiwa2_X = []
    q_waypoints = t.vector_values(np.linspace(t.start_time(), t.end_time(), 1000))
    for ii in range(q_waypoints.shape[1]):
        plant.SetPositions(plant_context, np.concatenate((q_waypoints[:, ii], np.zeros(14))))
        iiwa1_X.append(plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg1_start.model_instance)))
        iiwa2_X.append(plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body", wsg2_start.model_instance)))

    vertices = list(map(lambda X: X.translation(), iiwa1_X))
    colors = [np.array([0, 0, 1]) for _ in range(len(iiwa1_X))]
    vertices = np.stack(vertices).astype(np.float32).T
    colors = np.array(colors).astype(np.float32).T
    vis["paths"]["iiwa_1"].set_object(g.Points(g.PointsGeometry(vertices, color=colors),
                                    g.PointsMaterial(size=0.015)))
    vertices = list(map(lambda X: X.translation(), iiwa2_X))
    colors = [np.array([0, 0, 1]) for _ in range(len(iiwa2_X))]
    vertices = np.stack(vertices).astype(np.float32).T
    colors = np.array(colors).astype(np.float32).T
    vis["paths"]["iiwa_2"].set_object(g.Points(g.PointsGeometry(vertices, color=colors),
                                    g.PointsMaterial(size=0.015)))
    
    plant.SetPositions(plant_context, np.concatenate((q_waypoints[:, 0], q_waypoints[:, -1])))
    diagram.Publish(context)

In [ ]:
generate_segment_pics(trajectories, 0)

In [ ]:
generate_segment_pics(trajectories, 1)

In [ ]:
generate_segment_pics(trajectories, 2)